# Select a Date Range within the API

## Load the credentials from the `JSON` file

In [43]:
import json

with open('../../credentials.json', 'r') as f:    
    headers = json.load(f)

## Request the information of the indicator

### Choose the indicator

In [44]:
indicator = 84

url_indicator = f'https://api.esios.ree.es/indicators/{indicator}'
url_indicator

'https://api.esios.ree.es/indicators/84'

### How to introduce the date?

In [45]:
indicator = 84

url_indicator = f'https://api.esios.ree.es/indicators/{indicator}?start_date=2021-01-01&end_date=2021-12-31'
url_indicator

'https://api.esios.ree.es/indicators/84?start_date=2021-01-01&end_date=2021-12-31'

### How to introduce the API parameters through variables?

In [46]:
indicator = 84
start_date = '2020-01-01'
end_date = '2020-12-31'

url_indicator = f'https://api.esios.ree.es/indicators/{indicator}?start_date={start_date}&end_date={end_date}'
url_indicator

'https://api.esios.ree.es/indicators/84?start_date=2020-01-01&end_date=2020-12-31'

### Request the data

In [47]:
import requests

res = requests.get(url_indicator, headers=headers)
res

<Response [200]>

### Improved code

In [48]:
indicator = 84
url = f'https://api.esios.ree.es/indicators/{indicator}'

params = {
    'start_date': '2020-01-01',
    'end_date': '2020-12-31'
}

res = requests.get(url, params, headers=headers)
res

<Response [200]>

## Get the `JSON`

In [49]:
data = res.json()
data

{'indicator': {'name': 'Generación programada P48 Solar fotovoltaica',
  'short_name': 'Solar fotovoltaica',
  'id': 84,
  'composited': False,
  'step_type': 'step',
  'disaggregated': False,
  'magnitud': [{'name': 'Energía', 'id': 13}],
  'tiempo': [{'name': 'Hora', 'id': 223}],
  'geos': [{'geo_id': 8741, 'geo_name': 'Península'}],
  'values_updated_at': '2020-12-30T16:54:39.000+01:00',
  'values': [{'value': 22.4,
    'datetime': '2020-01-01T04:00:00.000+01:00',
    'datetime_utc': '2020-01-01T03:00:00Z',
    'tz_time': '2020-01-01T03:00:00.000Z',
    'geo_id': 8741,
    'geo_name': 'Península'},
   {'value': 17.1,
    'datetime': '2020-01-01T05:00:00.000+01:00',
    'datetime_utc': '2020-01-01T04:00:00Z',
    'tz_time': '2020-01-01T04:00:00.000Z',
    'geo_id': 8741,
    'geo_name': 'Península'},
   {'value': 21.8,
    'datetime': '2020-01-01T06:00:00.000+01:00',
    'datetime_utc': '2020-01-01T05:00:00Z',
    'tz_time': '2020-01-01T05:00:00.000Z',
    'geo_id': 8741,
    'geo_na

## Create a `DataFrame`

In [50]:
import pandas as pd

df = pd.DataFrame(data['indicator']['values'])
df = df.set_index('datetime')
df

,value,datetime_utc,tz_time,geo_id,geo_name
datetime,,,,,
2020-01-01T04:00:00.000+01:00,22.4,2020-01-01T03:00:00Z,2020-01-01T03:00:00.000Z,8741,Península
2020-01-01T05:00:00.000+01:00,17.1,2020-01-01T04:00:00Z,2020-01-01T04:00:00.000Z,8741,Península
...,...,...,...,...,...
2020-12-30T19:00:00.000+01:00,21.7,2020-12-30T18:00:00Z,2020-12-30T18:00:00.000Z,8741,Península
2020-12-30T20:00:00.000+01:00,0.3,2020-12-30T19:00:00Z,2020-12-30T19:00:00.000Z,8741,Península


## Automate data processing with functions

1. Open the previous exercise
2. Copy the cells from the `json`
3. Paste the cells below this one
4. Merge the cells
5. Indent lines to the right
6. Clean the code
    1. Comments together
    2. Remove unnecessary lines
7. Define the name of the function in the first line
8. Idenfity the variable the function needs as the ingredient and define the parameter
9. Identify the variable the function will return after executed and return the object at the end

In [52]:
def dataframe_p48(data):

    ## Create a `DataFrame`
    ### Get to the pattern of the data
    data_values = data['indicator']['values']

    ### Transform the `JSON` to a `DataFrame`
    df_solar = pd.DataFrame(data_values)

    ## Preprocess & clean the `DataFrame`
    ### Get the indicator name from metadata
    col_name = data['indicator']['short_name']

    df_solar = df_solar.rename({'value': col_name}, axis=1)

    ### Transform `datetime` to a `datetime`
    df_solar.datetime = pd.to_datetime(df_solar.datetime, format='ISO8601', utc=True).dt.tz_convert('Europe/Madrid')

    ### Select the columns `datetime` and `short_name` (previously)
    df_solar = df_solar[[col_name, 'datetime']].copy()

    ### Set the `datetime` as the index
    df_solar = df_solar.set_index('datetime')

    return df_solar

In [53]:
df = dataframe_p48(data)
df

,Solar fotovoltaica
datetime,
2020-01-01 04:00:00+01:00,22.4
2020-01-01 05:00:00+01:00,17.1
...,...
2020-12-30 19:00:00+01:00,21.7
2020-12-30 20:00:00+01:00,0.3


## Automate API call with a function

- Follow the same procedure we followed in the previous exercise

In [55]:
import requests

In [56]:
def get_indicator_data(indicator = 84, start_date = '2020-01-01', end_date = '2020-12-31'):
    
    url_indicator = f'https://api.esios.ree.es/indicators/{indicator}'
    
    params = {
        'start_date': start_date,
        'end_date': end_date
    }
    
    res = requests.get(url_indicator, params, headers=headers)
    data = res.json()
    
    return data

In [57]:
data_eolica = get_indicator_data(indicator=10010)
df_eolica = dataframe_p48(data=data_eolica)
df_eolica

,Eólica
datetime,
2020-01-01 00:00:00+01:00,1461.4
2020-01-01 01:00:00+01:00,1449.0
...,...
2020-12-30 23:00:00+01:00,10140.4
2020-12-31 00:00:00+01:00,10148.2


## Save the function in a portable module